***GENERATED CODE FOR telcochurnprediction PIPELINE***
***DON'T EDIT THIS CODE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'customerID', 'transformation_label': 'String Indexer'}], 'feature': 'customerID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '4245', 'mean': '', 'stddev': '', 'min': '0002-ORFBO', 'max': '9995-HOTOH', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'customerID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('customerID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'gender', 'transformation_label': 'String Indexer'}], 'feature': 'gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'Female', 'max': 'Male', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Partner', 'transformation_label': 'String Indexer'}], 'feature': 'Partner', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Partner', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Partner')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Dependents', 'transformation_label': 'String Indexer'}], 'feature': 'Dependents', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Dependents', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Dependents')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PhoneService', 'transformation_label': 'String Indexer'}], 'feature': 'PhoneService', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'PhoneService', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PhoneService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MultipleLines', 'transformation_label': 'String Indexer'}], 'feature': 'MultipleLines', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'MultipleLines', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MultipleLines')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'InternetService', 'transformation_label': 'String Indexer'}], 'feature': 'InternetService', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'DSL', 'max': 'No', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'InternetService', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('InternetService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OnlineSecurity', 'transformation_label': 'String Indexer'}], 'feature': 'OnlineSecurity', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'OnlineSecurity', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OnlineSecurity')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'OnlineBackup', 'transformation_label': 'String Indexer'}], 'feature': 'OnlineBackup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'OnlineBackup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('OnlineBackup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DeviceProtection', 'transformation_label': 'String Indexer'}], 'feature': 'DeviceProtection', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'DeviceProtection', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DeviceProtection')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TechSupport', 'transformation_label': 'String Indexer'}], 'feature': 'TechSupport', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'TechSupport', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TechSupport')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StreamingTV', 'transformation_label': 'String Indexer'}], 'feature': 'StreamingTV', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'StreamingTV', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StreamingTV')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'StreamingMovies', 'transformation_label': 'String Indexer'}], 'feature': 'StreamingMovies', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'StreamingMovies', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('StreamingMovies')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Contract', 'transformation_label': 'String Indexer'}], 'feature': 'Contract', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'Month-to-month', 'max': 'Two year', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Contract', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Contract')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PaperlessBilling', 'transformation_label': 'String Indexer'}], 'feature': 'PaperlessBilling', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'PaperlessBilling', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PaperlessBilling')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PaymentMethod', 'transformation_label': 'String Indexer'}], 'feature': 'PaymentMethod', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'Bank transfer (automatic)', 'max': 'Mailed check', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'PaymentMethod', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PaymentMethod')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TotalCharges', 'transformation_label': 'String Indexer'}], 'feature': 'TotalCharges', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '2271.62', 'stddev': '2273.64', 'min': ' ', 'max': '999.8', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'TotalCharges', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TotalCharges')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Churn', 'transformation_label': 'String Indexer'}], 'feature': 'Churn', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '4245', 'mean': '', 'stddev': '', 'min': 'No', 'max': 'Yes', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}]}, {'feature_label': 'Churn', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Churn')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': sparkDF.columns}
    return data


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

%run telcochurnpredictionHooks.ipynb
try:
	sourcePreExecutionHook()

	telcochurnpredictiondbfs = DBFSConnector.fetch(spark, "{'url': '/Demo/TelcoChurnPredictionTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

	sourcePostExecutionHook(telcochurnpredictiondbfs)

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
%run telcochurnpredictionHooks.ipynb
try:
	transformationPreExecutionHook()

	telcochurnpredictionautofe = TransformationMain.run(telcochurnpredictiondbfs,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "customerID", "transformation_label": "String Indexer"}], "feature": "customerID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "0002-ORFBO", "max": "9995-HOTOH", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "gender", "transformation_label": "String Indexer"}], "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "Female", "max": "Male", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "SeniorCitizen", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4245", "mean": "0.17", "stddev": "0.38", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": [{"feature_label": "Partner", "transformation_label": "String Indexer"}], "feature": "Partner", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Dependents", "transformation_label": "String Indexer"}], "feature": "Dependents", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "tenure", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "4245", "mean": "32.33", "stddev": "24.66", "min": "0", "max": "72", "missing": "0"}}, {"transformationsData": [{"feature_label": "PhoneService", "transformation_label": "String Indexer"}], "feature": "PhoneService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "MultipleLines", "transformation_label": "String Indexer"}], "feature": "MultipleLines", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "InternetService", "transformation_label": "String Indexer"}], "feature": "InternetService", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "DSL", "max": "No", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "OnlineSecurity", "transformation_label": "String Indexer"}], "feature": "OnlineSecurity", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "OnlineBackup", "transformation_label": "String Indexer"}], "feature": "OnlineBackup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "DeviceProtection", "transformation_label": "String Indexer"}], "feature": "DeviceProtection", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "TechSupport", "transformation_label": "String Indexer"}], "feature": "TechSupport", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "StreamingTV", "transformation_label": "String Indexer"}], "feature": "StreamingTV", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "StreamingMovies", "transformation_label": "String Indexer"}], "feature": "StreamingMovies", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Contract", "transformation_label": "String Indexer"}], "feature": "Contract", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "Month-to-month", "max": "Two year", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "PaperlessBilling", "transformation_label": "String Indexer"}], "feature": "PaperlessBilling", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "PaymentMethod", "transformation_label": "String Indexer"}], "feature": "PaymentMethod", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "Bank transfer (automatic)", "max": "Mailed check", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MonthlyCharges", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "4245", "mean": "64.39", "stddev": "30.18", "min": "18.4", "max": "118.75", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "TotalCharges", "transformation_label": "String Indexer"}], "feature": "TotalCharges", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "2271.62", "stddev": "2273.64", "min": " ", "max": "999.8", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"transformationsData": [{"feature_label": "Churn", "transformation_label": "String Indexer"}], "feature": "Churn", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "4245", "mean": "", "stddev": "", "min": "No", "max": "Yes", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}]}, {"feature": "customerID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "4245", "mean": "2122.0", "stddev": "1225.57", "min": "0.0", "max": "4244.0", "missing": "0"}}, {"feature": "gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.49", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}}, {"feature": "Partner_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.48", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}}, {"feature": "Dependents_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.29", "stddev": "0.46", "min": "0", "max": "1", "missing": "0"}}, {"feature": "PhoneService_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.1", "stddev": "0.3", "min": "0", "max": "1", "missing": "0"}}, {"feature": "MultipleLines_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.62", "stddev": "0.66", "min": "0", "max": "2", "missing": "0"}}, {"feature": "InternetService_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.79", "stddev": "0.78", "min": "0", "max": "2", "missing": "0"}}, {"feature": "OnlineSecurity_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.72", "stddev": "0.8", "min": "0", "max": "2", "missing": "0"}}, {"feature": "OnlineBackup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.79", "stddev": "0.78", "min": "0", "max": "2", "missing": "0"}}, {"feature": "DeviceProtection_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.79", "stddev": "0.78", "min": "0", "max": "2", "missing": "0"}}, {"feature": "TechSupport_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.73", "stddev": "0.8", "min": "0", "max": "2", "missing": "0"}}, {"feature": "StreamingTV_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.82", "stddev": "0.77", "min": "0", "max": "2", "missing": "0"}}, {"feature": "StreamingMovies_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.83", "stddev": "0.76", "min": "0", "max": "2", "missing": "0"}}, {"feature": "Contract_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.66", "stddev": "0.8", "min": "0", "max": "2", "missing": "0"}}, {"feature": "PaperlessBilling_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.41", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}}, {"feature": "PaymentMethod_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "1.33", "stddev": "1.15", "min": "0", "max": "3", "missing": "0"}}, {"feature": "TotalCharges_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "4245", "mean": "1918.47", "stddev": "1212.1", "min": "0.0", "max": "4032.0", "missing": "0"}}, {"feature": "Churn_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "4245", "mean": "0.27", "stddev": "0.44", "min": "0", "max": "1", "missing": "0"}}]}))

	transformationPostExecutionHook(telcochurnpredictionautofe)

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
%run telcochurnpredictionHooks.ipynb
try:
	mlPreExecutionHook()

	dataAutoML=functionClassification(telcochurnpredictionautofe, ["SeniorCitizen", "tenure", "MonthlyCharges", "customerID_stringindexer", "gender_stringindexer", "Partner_stringindexer", "Dependents_stringindexer", "PhoneService_stringindexer", "MultipleLines_stringindexer", "InternetService_stringindexer", "OnlineSecurity_stringindexer", "OnlineBackup_stringindexer", "DeviceProtection_stringindexer", "TechSupport_stringindexer", "StreamingTV_stringindexer", "StreamingMovies_stringindexer", "Contract_stringindexer", "PaperlessBilling_stringindexer", "PaymentMethod_stringindexer", "TotalCharges_stringindexer"], "Churn_stringindexer")

	mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


**PREDICT ON TRAINED MODEL**

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

